# Laboratory 1: setting up Twitter API

## Set up

In [ ]:
import os
os.environ['TOKEN'] = #BEARER TOKEN HERE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Import libraries

In [ ]:
import requests 
import pandas as pd 
import time

If you need to download a library, use the following code, just specify the name of the library you need (here we downloaded emoji library)

In [ ]:
!pip install emoji

### Set up headers

In [ ]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [ ]:
headers = create_headers(os.environ['TOKEN'])

## Download data

Date format and other parameter explanations available here:

https://developer.twitter.com/en/docs/twitter-api/premium/search-api/api-reference/premium-search#DataParameters

In [ ]:
def create_url(keyword, start_date, end_date, env_label, endpoint="fullarchive"):
    
    search_url = "https://api.twitter.com/1.1/tweets/search/{}.json".format(endpoint+"/"+env_label) 

    #change params based on the endpoint you are using
    query_params = {'query': keyword, 'fromDate': start_date, 'toDate': end_date}
    return (search_url, query_params)

In [ ]:
def connect_to_endpoint(url, headers, params, next_token = None):
    if next_token is not None and next_token != '':
      params['next'] = next_token
    response = requests.request("GET", url, headers = headers, params = params)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [ ]:
def get_data(keyword, start_time, end_time, next_token, env_label, endpoint):
  results = []
  
  while next_token is not None:
    ##this part here for one request
    url = create_url(keyword, start_time,end_time, env_label, endpoint)
    json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
    
    if "results" in json_response:
      results.extend(json_response["results"])
    ### up until this point
    if "next" in json_response:
        next_token = json_response["next"]
    else:
      next_token = None
    time.sleep(1)
  
  return results

In [ ]:
def get_single_response(keyword, start_time, end_time, env_label, endpoint):
  #endpoint can be fullarchive or 30day
  results = []
  url = create_url(keyword, start_time,end_time, env_label, endpoint)
  json_response = connect_to_endpoint(url[0], headers, url[1])
  
  if "results" in json_response:
    results.extend(json_response["results"])
  
  return results

In [ ]:
#if you are using the 30day endpoint, make sure you specify dates that are within 30day range!
tweets = get_single_response("COVID19 lang:en", "202110080000", "202111070000", "30day", "30day")

### Inspect data

In [ ]:
len(tweets)

In [ ]:
tweets[0]

# Laboratory 2: working with Twitter data

First, we want to convert the data into Pandas DataFrame. This format enables us easy manipulation of the data as well as saving/loading data.

Since we have our tweets saved as a list of dictionaries, we can easily convert it to DataFrame by executing the cell blow.

In [ ]:
tweets_df = pd.DataFrame(tweets)

In [ ]:
tweets_df

### Saving the results

Once we have our Tweets as a DataFrame it is a good idea to save it on the disk. 

Be mindful of the fact that the storage of a Colab notebook is deleted everytime runtime is interrupted or restarted, so you need to manually download it to your computer or mount your Google Drive and save it there (this option is unavailable if you're using university's email account for Drive).

In [ ]:
path = "/content/" #enter the path to your Drive or leave this as default

We can save it as a comma-separated values file, which enables opening it in a spreadsheet editor and inspecting it.

In [ ]:
tweets_df.to_csv(path+"tweets.csv", index=False)

In order to preserve datatypes, we should save it as a parquet or pickle file.

In [ ]:
tweets_df.to_pickle(path+"tweets.pkl")

### Loading the data

If you want to load the results you have previously saved, simply execute the next code, specifying the path to the file.

You will need to either upload it to the Colab workspace or copy the path to the file on Drive.

In [ ]:
tweets_df = pd.read_pickle(path+"tweets.pkl")

In [ ]:
# if tweets are uploaded in colab

tweets_df = pd.read_pickle("tweets.pkl")

In [ ]:
tweets_df

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,is_quote_status,quote_count,reply_count,retweet_count,favorite_count,entities,favorited,retweeted,filter_level,lang,matching_rules,extended_tweet,possibly_sensitive,display_text_range,extended_entities
0,Sat Nov 06 23:59:58 +0000 2021,1457135701732904962,1457135701732904962,RT @mansukhmandviya: Appreciate @Murugan_MoS j...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,None,"{'id': 1366620482997420034, 'id_str': '1366620...",None,None,None,None,{'created_at': 'Sat Nov 06 16:23:21 +0000 2021...,1.456847e+18,1456847227930890241,{'created_at': 'Sat Nov 06 04:53:40 +0000 2021...,"{'url': 'https://t.co/0gTV7O4gcB', 'expanded':...",True,0,0,0,0,"{'hashtags': [{'text': 'HarGharDastak', 'indic...",False,False,low,en,[{'tag': None}],NaN,NaN,NaN,NaN
1,Sat Nov 06 23:59:57 +0000 2021,1457135697337458691,1457135697337458691,RT @StephenSteglik: Shoutout to @BigBird for n...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,None,"{'id': 38947085, 'id_str': '38947085', 'name':...",None,None,None,None,{'created_at': 'Sat Nov 06 23:59:24 +0000 2021...,1.456972e+18,1456971880666046465,{'created_at': 'Sat Nov 06 13:09:00 +0000 2021...,"{'url': 'https://t.co/r58ZIiZC90', 'expanded':...",True,0,0,0,0,"{'hashtags': [{'text': 'GetVaccinated', 'indic...",False,False,low,en,[{'tag': None}],NaN,NaN,NaN,NaN
2,Sat Nov 06 23:59:56 +0000 2021,1457135694367924227,1457135694367924227,RT @apsmunro: If you hear there have been &gt;...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,None,"{'id': 16078191, 'id_str': '16078191', 'name':...",None,None,None,None,{'created_at': 'Sat Nov 06 12:36:15 +0000 2021...,NaN,NaN,NaN,NaN,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,[{'tag': None}],NaN,NaN,NaN,NaN
3,Sat Nov 06 23:59:55 +0000 2021,1457135690093776899,1457135690093776899,Tonight on CHEK News at 5 p.m. with @JasmineBa...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",True,NaN,None,NaN,None,None,"{'id': 156749387, 'id_str': '156749387', 'name...",None,None,None,None,NaN,NaN,NaN,NaN,NaN,False,0,1,2,2,"{'hashtags': [{'text': 'COVID19', 'indices': [...",False,False,low,en,[{'tag': None}],{'full_text': 'Tonight on CHEK News at 5 p.m. ...,False,NaN,NaN
4,Sat Nov 06 23:59:51 +0000 2021,1457135673773727748,1457135673773727748,RT @SovernNation: A #COVID19 outbreak among th...,"<a href=""https://about.twitter.com/products/tw...",False,NaN,None,NaN,None,None,"{'id': 1295326471821520899, 'id_str': '1295326...",None,None,None,None,{'created_at': 'Sat Nov 06 23:59:33 +0000 2021...,NaN,NaN,NaN,NaN,False,0,0,0,0,"{'hashtags': [{'text': 'COVID19', 'indices': [...",False,False,low,en,[{'tag': None}],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Sat Nov 06 23:55:43 +0000 2021,1457134631937929221,1457134631937929221,RT @disclosetv: NEW - Sesame Street with Ameri...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,None,"{'id': 3233164824, 'id_str': '3233164824', 'na...",None,None,None,None,{'created_at': 'Sat Nov 06 12:45:38 +0000 2021...,NaN,NaN,NaN,NaN,False,0,0,0,0,"{'hashtags': [{'text': 'COVID19', 'indices': [...",False,False,low,en,[{'tag': None}],NaN,False,NaN,"{'media': [{'id': 1456965743774474244, 'id_str..."
96,Sat Nov 06 23:55:43 +0000 2021,1457134631392665602,1457134631392665602,RT @UNICEFIndia: As we celebrate the #COVID19 ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,None,"{'id': 1407216867551244288, 'id_str': '1407216...",None,None,None,None,{'created_at': 'Tue Oct 

### Preprocessing the data

In our dataframe we have the entire Tweet object. Some columns that might be of particular interest to us are: 

*   created_at - date when Tweet was posted
*   id/id_str - unique Tweet identifiers
*   text - the content of the Tweet
*   user - information about the user who posted the Tweet
*   retweeted_status  - information about the original Tweet
*   quote/reply/retweet/favorite count - Tweet metrics
*   entities - hashtags, urls, user_mentions present in Tweet

We can filter the dataframe and keep only columns we are interested in. You can pick which columns you'd like to keep and put them int the column_list below.



In [ ]:
tweets_filtered = tweets_df.copy() #it's a good idea to work on the copy of original dataframe, so we can always go back to it if we mess something up
column_list = ["created_at", "id_str", "text", "user", "retweeted_status", "quote_count", "reply_count", "retweet_count", "favorite_count", "entities"]
tweets_filtered = tweets_filtered[column_list]

In [ ]:
tweets_filtered

,created_at,id_str,text,user,retweeted_status,quote_count,reply_count,retweet_count,favorite_count,entities
0,Sat Nov 06 23:59:58 +0000 2021,1457135701732904962,RT @mansukhmandviya: Appreciate @Murugan_MoS j...,"{'id': 1366620482997420034, 'id_str': '1366620...",{'created_at': 'Sat Nov 06 16:23:21 +0000 2021...,0,0,0,0,"{'hashtags': [{'text': 'HarGharDastak', 'indic..."
1,Sat Nov 06 23:59:57 +0000 2021,1457135697337458691,RT @StephenSteglik: Shoutout to @BigBird for n...,"{'id': 38947085, 'id_str': '38947085', 'name':...",{'created_at': 'Sat Nov 06 23:59:24 +0000 2021...,0,0,0,0,"{'hashtags': [{'text': 'GetVaccinated', 'indic..."
2,Sat Nov 06 23:59:56 +0000 2021,1457135694367924227,RT @apsmunro: If you hear there have been &gt;...,"{'id': 16078191, 'id_str': '16078191', 'name':...",{'created_at': 'Sat Nov 06 12:36:15 +0000 2021...,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ..."
3,Sat Nov 06 23:59:55 +0000 2021,1457135690093776899,Tonight on CHEK News at 5 p.m. with @JasmineBa...,"{'id': 156749387, 'id_str': '156749387', 'name...",NaN,0,1,2,2,"{'hashtags': [{'text': 'COVID19', 'indices': [..."
4,Sat Nov 06 23:59:51 +0000 2021,1457135673773727748,RT @SovernNation: A #COVID19 outbreak among th...,"{'id': 1295326471821520899, 'id_str': '1295326...",{'created_at': 'Sat Nov 06 23:59:33 +0000 2021...,0,0,0,0,"{'hashtags': [{'text': 'COVID19', 'indices': [..."
...,...,...,...,...,...,...,...,...,...,...
95,Sat Nov 06 23:55:43 +0000 2021,1457134631937929221,RT @disclosetv: NEW - Sesame Street with Ameri...,"{'id': 3233164824, 'id_str': '3233164824', 'na...",{'created_at': 'Sat Nov 06 12:45:38 +0000 2021...,0,0,0,0,"{'hashtags': [{'text': 'COVID19', 'indices': [..."
96,Sat Nov 06 23:55:43 +0000 2021,1457134631392665602,RT @UNICEFIndia: As we celebrate the #COVID19 ...,"{'id': 1407216867551244288, 'id_str': '1407216...",{'created_at': 'Tue Oct 19 03:30:00 +0000 2021...,0,0,0,0,"{'hashtags': [{'text': 'COVID19', 'indices': [..."
97,Sat Nov 06 23:55:38 +0000 2021,1457134610958139398,RT @BulliedAutistic: @Tammlin1963 You can't tr...,"{'id': 1213639608304431104, 'id_str': '1213639...",{'created_at': 'Sat Nov 06 23:55:27 +0000 2021...,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ..."
98,Sat Nov 06 23:55:35 +0000 2021,1457134599432282114,@CoxeAnne @LovesBulldawgs @AnarchistGolden @Do...,"{'id': 1368049995534974976, 'id_str': '1368049...",NaN,0,0,0,0,"{'hashtags': [], 'urls': [{'url': 'https://t.c..."


## Extracting words/hashtags

There are many ways to build networks from the data we download from Twitter.

One possibility is to have a bipartite network of Tweets and words/hashtags and then observe word, hashtag or word-hashtag projections.

### Extracting words

In order to extract words, we first need to clean the Tweet text. This way we will remove punctuation, hashtags/mentions/urls (they are preserved in the entity column anyway). We will also turn all letters to lowercase.

You can also consider removing stopwords, removing words that are not in the english language corpora, lematizing the words, etc. I suggest you research nltk library and its possibilities.

In [ ]:
import re
import string

In [ ]:
def cleaner(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) # remove mentions
    tweet = re.sub("#[A-Za-z0-9]+", "",tweet) # remove hashtags
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) # remove http links
    tweet = " ".join(tweet.split())
    tweet = str.lower(tweet) #to lowercase
    table = str.maketrans(dict.fromkeys(string.punctuation)) 
    tweet = tweet.translate(table)# remove punctuation         
    return tweet

In [ ]:
tweets_filtered["clean_text"] = tweets_filtered["text"].map(cleaner)
# tweets_filtered['clean_text']

We are going to loop through the dataframe and then through the words in the clean text. We are going to add the words as keys to dictionary and use their frequencies as values.

In [ ]:
#initialize an empty dict
unique_words = {}
for row in tweets_filtered.clean_text:
  for word in row.split(" "):
    #if the word is encountered for the first time add to dict as key and set its value to 0
    unique_words.setdefault(word,0)
    #increase the value (i.e the count) of the word by 1 every time it is encountered
    unique_words[word] += 1

In [ ]:
#remove empty word
unique_words.pop("")
#remove word 'rt'
unique_words.pop("rt")

63

In [ ]:
# sorted(unique_words)

We can inspect the words as a dataframe. 


You can always save this dataframe as .csv for future reference.

In [ ]:
uw_df = pd.DataFrame.from_dict(unique_words, orient='index').reset_index()
uw_df.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
uw_df.sort_values(by=['Count'], ascending=False, inplace=True)
uw_df

,Word,Count
55,the,61
4,to,37
35,is,27
16,a,26
9,of,24
...,...,...
314,flu,1
313,every,1
312,halfthere,1
309,comment,1


### Extracting the hashtags

We are going to loop through the dataframe and then through the hashtags in the entities. We are going to add the hashtags as keys to dictionary and use their frequencies as values. At the same time, we are going to save them in a list and add them to a separate column to facilitate our future work.

In [ ]:
unique_hashtags = {}

tweets_filtered["hashtags"] = ""

for idx, row in tweets_filtered.iterrows():
  hashtag_list = []
  for hashtag in row["entities"]["hashtags"]:
    unique_hashtags.setdefault("#"+hashtag["text"], 0)
    unique_hashtags['#'+hashtag["text"]] += 1
    hashtag_list.append(hashtag["text"])
  tweets_filtered.at[idx,"hashtags"] = hashtag_list

In [ ]:
uh_df = pd.DataFrame.from_dict(unique_hashtags, orient='index').reset_index()
uh_df.rename(columns = {'index':'Hashtag', 0:'Count'}, inplace=True)
uh_df.sort_values(by=['Count'], ascending=False, inplace=True)

In [ ]:
uh_df

,Hashtag,Count
3,#COVID19,34
2,#Covid19,5
20,#Vaccine,4
22,#CCP,3
25,#covid19,2
1,#GetVaccinated,2
9,#auspol,2
30,#AI,2
31,#leadership,1
26,#abstract,1


## Building the network

We are going to use the networkx library, which is a Python library that enables network science analysis of the data.

We are going to use it to create our network and extract edgelist from it, since we can easily import it to Gephi (a software we are going to see in visualization labs).

However, it offers implemented algorithms for analysis (for example PageRank) that you can use out-of-box to analyze your network.

But first, we will loop through our dataframe and connect words and hashtags if they appear together in the same Tweet.

In [ ]:
import itertools
import networkx as nx

In [ ]:
uh = unique_hashtags.keys()
uw = unique_words.keys()  

In [ ]:
network = {}
network_key = 0
for index, row in tweets_filtered.iterrows():
    #hashtags extracted from Tweet do not have the # sign in front of them but we will add it to differentiate hashtags from words
    combined_list = ['#'+hashtag for hashtag in row["hashtags"] if '#'+hashtag in unique_hashtags] + [word for word in str.split(row["clean_text"], " ") if word in uw]
    #itertool product creates Cartesian product of each element in the combined list
    for pair in itertools.product(combined_list, combined_list):
        #exclude self-loops and count each pair only once because our graph is undirected and we do not take self-loops into account
        if pair[0]!=pair[1] and not(pair[::-1] in network):
            network.setdefault(pair,0)
            network[pair] += 1 
    
network_df = pd.DataFrame.from_dict(network, orient="index")

In [ ]:
network_df.reset_index(inplace=True)
network_df.columns = ["pair","weight"]
network_df.sort_values(by="weight",inplace=True, ascending=False)
network_df

,pair,weight
1038,"(the, to)",26
510,"(#COVID19, the)",24
1947,"(the, this)",17
1307,"(covid19, the)",16
352,"(is, a)",16
...,...,...
3591,"(chronic, sack)",1
3590,"(chronic, move)",1
3589,"(chronic, the)",1
3588,"(chronic, feared)",1


In [ ]:
#to get weighted graph we need a list of 3-element tuplels (u,v,w) where u and v are nodes and w is a number representing weight
up_weighted = []
for edge in network:
    #we can filter edges by weight by uncommenting the next line and setting desired weight threshold
    #if(network[edge])>1:
    up_weighted.append((edge[0],edge[1],network[edge]))

G = nx.Graph()
G.add_weighted_edges_from(up_weighted)

In [ ]:
print(len(G.nodes()))
print(len(G.edges()))

796
9856


#### SAVE EDGELIST

In [ ]:
filename = "./edgelist.csv"

In [ ]:
nx.write_weighted_edgelist(G, filename, delimiter=",")

In [ ]:
#add header with appropriate column names (works on collab and Linux/Mac(?))
! sed -i.bak 1i"Source,Target,Weight" ./edgelist.csv

### Create Node List


In [ ]:
word_nodes = pd.DataFrame.from_dict(unique_words,orient="index")
word_nodes.reset_index(inplace=True)
word_nodes["Label"] = word_nodes["index"]
word_nodes.rename(columns={"index":"Id",0:"delete"},inplace=True)
word_nodes = word_nodes.drop(columns=['delete'])

word_nodes

,Id,Label
0,appreciate,appreciate
1,mos,mos
2,jis,jis
3,initiative,initiative
4,to,to
...,...,...
749,attention,attention
750,science,science
751,ya,ya
752,else,else


In [ ]:
hashtag_nodes = uh_df.copy()
hashtag_nodes["Label"] = hashtag_nodes["Hashtag"]
hashtag_nodes.rename(columns={"Hashtag":"Id"},inplace=True)
hashtag_nodes = hashtag_nodes.drop(columns=['Count'])
hashtag_nodes

,Id,Label
3,#COVID19,#COVID19
2,#Covid19,#Covid19
20,#Vaccine,#Vaccine
22,#CCP,#CCP
25,#covid19,#covid19
1,#GetVaccinated,#GetVaccinated
9,#auspol,#auspol
30,#AI,#AI
31,#leadership,#leadership
26,#abstract,#abstract


#### SAVE NODELIST

In [ ]:
nodelist = hashtag_nodes.append(word_nodes, ignore_index=True)

nodelist.to_csv("nodelist.csv",index=False)

Tasks: 

*   Extract username of user who posted the tweet into a column "screen_name". Follow the procedure we used to get the hashtags.
*   Create a network of users using the mention relation. Is this a directed or undirected graph?
*   We created a network where nodes are mixed (both words and hashtags). Create network of words only and one of hashtags only.
* Pick one of these network and rank the nodes using PageRank centrality. Extract information about top-20 rated nodes.





In [ ]:
tweets_filtered['user'][0]['name']

'G SANKAR'

In [ ]:
# Extract username

unique_username = {}

tweets_filtered["screen_name"] = ""

for idx, row in tweets_filtered.iterrows():
  username_list = []
  for username in row["user"]:
    unique_username.setdefault(username['name'], 0)
    unique_username[username['name']] += 1
    username_list.append(username['name'])
  tweets_filtered.at[idx,"screen_name"] = username_list


# Unique username dataframe
uh_df = pd.DataFrame.from_dict(unique_username, orient='index').reset_index()
uh_df.rename(columns = {'index':'screen_name', 0:'Count'}, inplace=True)
uh_df.sort_values(by=['Count'], ascending=False, inplace=True)

TypeError: ignored